# Leaflet cluster map of talk locations

Assuming you are working in a Linux or Windows Subsystem for Linux environment, you may need to install some dependencies. Assuming a clean installation, the following will be needed:

```bash
sudo apt install jupyter
sudo apt install python3-pip
pip install python-frontmatter getorg --upgrade
```

After which you can run this from the `_talks/` directory, via:

```bash
 jupyter nbconvert --to notebook --execute talkmap.ipynb --output talkmap_out.ipynb
```
 
The `_talks/` directory contains `.md` files of all your talks. This scrapes the location YAML field from each `.md` file, geolocates it with `geopy/Nominatim`, and uses the `getorg` library to output data, HTML, and Javascript for a standalone cluster map.

In [1]:
# Start by installing the dependencies
!pip install python-frontmatter getorg --upgrade
import frontmatter
import glob
import getorg
from geopy import Nominatim
from geopy.exc import GeocoderTimedOut

Defaulting to user installation because normal site-packages is not writeable
Iywidgets and ipyleaflet support disabled. You must be in a Jupyter notebook to use this feature.
Error raised:
No module named 'ipywidgets'
Check that you have enabled ipyleaflet in Jupyter with:
    jupyter nbextension enable --py ipyleaflet


In [2]:
# Collect the Markdown files
g = glob.glob("_talks/*.md")

In [3]:
# Set the default timeout, in seconds
TIMEOUT = 5

# Prepare to geolocate
geocoder = Nominatim(user_agent="academicpages.github.io")
location_dict = {}
location = ""
permalink = ""
title = ""

In the event that this times out with an error, double check to make sure that the location is can be properly geolocated.

In [4]:
# Perform geolocation
for file in g:
    # Read the file
    data = frontmatter.load(file)
    data = data.to_dict()

    # Press on if the location is not present
    if 'location' not in data:
        continue

    # Prepare the description
    title = data['title'].strip()
    venue = data['venue'].strip()
    location = data['location'].strip()
    description = f"{title}<br />{venue}; {location}"

    # Geocode the location and report the status
    try:
        location_dict[description] = geocoder.geocode(location, timeout=TIMEOUT)
        print(description, location_dict[description])
    except ValueError as ex:
        print(f"Error: geocode failed on input {location} with message {ex}")
    except GeocoderTimedOut as ex:
        print(f"Error: geocode timed out on input {location} with message {ex}")
    except Exception as ex:
        print(f"An unhandled exception occurred while processing input {location} with message {ex}")

Vector Boson Fusion and Scattering<br />EPFL Sinergia Meeting; Lausanne, Switzerland Lausanne, District de Lausanne, Vaud, Schweiz/Suisse/Svizzera/Svizra
Fully differential VBFH at NNLO<br />EPSHEP 2015; Vienna, Austria Wien, Österreich
VBF Higgs production at NNLO ... and beyond<br />UZH Theoretical Particle Physics Seminar; Zurich, Switzerland Zürich, Bezirk Zürich, Zürich, Schweiz/Suisse/Svizzera/Svizra
Parton Shower Effects in Vector Boson Fusion<br />LHC Higgs Cross Section Working Group Meeting; Online Online, Croix d'Argent, Montpellier, Hérault, Occitanie, France métropolitaine, France
NNLOPS accurate Drell-Yan production<br />Cargese Summer School 2014; Cargèse, France Cargèse, Ajaccio, Corse-du-Sud, Corse, France métropolitaine, 20130, France
VBF Higgs Production at the HL-/HE-LHC<br />HL/HE-LHC WG1 Meeting – Electroweak physics; Geneva, Switzerland Genève, Schweiz/Suisse/Svizzera/Svizra
QCD for the LHC<br />Nordic Winter School on Cosmology and Particle Physics; Svingvoll, N

In [5]:
# Save the map
m = getorg.orgmap.create_map_obj()
getorg.orgmap.output_html_cluster_map(location_dict, folder_name="talkmap", hashed_usernames=False)

'Written map to talkmap/'